# Step 1: Install Python packages

In [2]:
import os
import html5lib
import pandas as pd
from selenium import webdriver                   
from selenium.webdriver.common.keys import Keys   
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from datetime import date, timedelta, datetime as dt
from bs4 import BeautifulSoup as bs    

# Step 2: Preparation

In [3]:
class RemoteDriverStartService():
    options = webdriver.ChromeOptions()
    # Set user app data to a new directory
    options.add_argument("user-data-dir=C:\\Users\\Donley\\App Data\\Google\\Chrome\\Application\\User Data\\Kit")
    options.add_experimental_option("Proxy", "null")
    options.add_experimental_option("excludeSwitches", ["ignore-certificate-errors"])
    # Create a download path for external data sources as default: 
    options.add_experimental_option("prefs", {
      "download.default_directory": r"C:\Users\Donley\Documents\GA_TECH\SUBMISSIONS\PROJECT2-CHALLENGE\data\external",
      "download.prompt_for_download": False,
      "download.directory_upgrade": True,
      "safebrowsing.enabled": True
    }),
    # Add those optional features to capabilities
    caps = options.to_capabilities()  
    def start_driver(self):
        return webdriver.Remote(command_executor='http://127.0.0.1:4444', 
                                desired_capabilities=self.caps)
# Set class equal to new capabilities:
DesiredCapabilities = RemoteDriverStartService()  

In [4]:
# Create variables for scraping: 
wsj = "https://www.wsj.com/market-data/stocks?mod=nav_top_subsection"
# Download data to paths, csv's, json, etc: 
    # for external data sources
external = "../data/external/"
    # for processed data sources with ID's
processed = "../data/processed/"

In [5]:
# Locate Driver in system
current_path = os.getcwd()

# save the .exe file under the same directory of the web-scrape python script.
Path = os.path.join(current_path, "chromedriver")

# Initialize Chrome driver and start browser session controlled by automated test software under Kit profile.
caps = webdriver.DesiredCapabilities.CHROME.copy()
caps['acceptInsecureCerts'] = True
# caps = webdriver.DesiredCapabilities.CHROME.copy()
# caps['acceptInsecureCerts'] = True
# driver = webdriver.Chrome(options=options, desired_capabilities=caps)
driver = webdriver.Chrome(executable_path='chromedriver', desired_capabilities=caps)

# Get the URL
driver.get(wsj)

# Step 3: Find the IDs of the items we want to scrape for

In [6]:
# Give it time to search for ID and allow the page time to load:
timeout = 30
try:
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.ID, "root")))
except TimeoutException:
    driver.quit()
    

# Step 4: Techniques to make more human-like web-scrapers

In [7]:
# If the website detects us as a web-scraper, it will cut our connection so we cannot pull more data and have to re-start our scraper. This largely impacts the efficiency of the scraper and involves a lot of manual interference. There are a few techniques we can use to make the scraper more human-like:
# (1) Randomize the sleep time
# This can be easily implemented as below wherever needed:
#sleep for sometime between 5 and 8 seconds
# time.sleep(random.uniform(5,8))
# (2) Randomize the user agent for the web browser
# This is also easy and can be added to the browser options as below:
# ua = UserAgent()
# userAgent = ua.random
# Firefox_options = webdriver.FirefoxOptions()
# Firefox_options.add_argument(f’user-agent={userAgent}’)
# browser = webdriver.Firefox(executable_path = DRIVER_BIN, options=Firefox_options)
# (3) Use dynamic proxy/IP
# This requires more work than the above two. Usually free proxies are not stable and most of them don’t respond to requests, so we need to first a free proxy that responds to our requests. This website (also named as “url” in the script below) provides a lot of free proxies which we scrape down for our use. We will use Python BeautifulSoup package to scrape a list of proxies, and use Python requests package to test whether the proxy responds to our requests to the link.
# def get_proxy(link):
#     url = "https://www.sslproxies.org/"
#     r = requests.get(url)
#     soup = BeautifulSoup(r.content, 'html5lib')
#     proxies_list = list(map(lambda x: x[0]+':'+x[1], list(zip(map(lambda x: x.text, soup.findAll('td')[::8]), map(lambda x: x.text, soup.findAll('td')[1::8])))))
#     while 1:
#         try:
#             selected_ip = choice(proxies_list)
#             proxy = {'https': selected_ip, 'http': selected_ip}
#             headers = {'User-Agent': ua.random}
#             print('Using proxy:{}'.format(proxy))
#             r = requests.request('get', link, proxies=proxy, headers=headers, timeout=5)
#             break
#         except:
#             pass
        
#     return proxy
# We then add the working proxy to the browser option, similar to how we added the fake user agent:
# link = "https://www.expedia.com"
# proxy = get_proxy(link)
# Firefox_options.add_argument('--proxy-server=%s' % proxy)
# browser = webdriver.Firefox(executable_path = DRIVER_BIN, options=Firefox_options)

# Step 5: The full code that runs the scraper and save the data to .csv files

In [8]:
# Read Html to generate S&P 500 table: 
table = driver.find_element_by_id("root").get_attribute('outerHTML')
tables  = pd.read_html(table)
tables

[              Unnamed: 0      Last     Chg  %Chg
 0                   DJIA  28303.46  530.70  1.91
 1       Nasdaq Composite  11364.60  210.00  1.88
 2                S&P 500   3419.45   58.50  1.74
 3  DJ Total Stock Market  35104.71  612.56  1.78
 4           Russell 2000   1611.04   33.75  2.14
 5         NYSE Composite  13042.33  204.45  1.59
 6           Barron's 400    745.74   13.03  1.78
 7        CBOE Volatility     28.06   -1.42 -4.82
 8           DJIA Futures  28210.00   31.00  0.11
 9        S&P 500 Futures   3410.00    3.20  0.09,
   Unnamed: 0_level_0               NYSE             Nasdaq
               Issues Unnamed: 1_level_1 Unnamed: 2_level_1
            Issues At Unnamed: 1_level_2 Unnamed: 2_level_2
         Share Volume Unnamed: 1_level_3 Unnamed: 2_level_3
 0          Advancing               2219               2678
 1          Declining                817                798
 2          Unchanged                 90                 82
 3              Total        

In [9]:
# Generate List of tables on page:
wsj_tables =[df for df in tables]

In [10]:
# Get table 5 and rename columns:
sp500_sectors_df = wsj_tables[5]
sp500_sectors_df.columns = ["S&P 500 & Sectors", "% Change"]
sp500_sectors_df.reset_index(drop=True, inplace=True)
sp500_sectors_df

,S&P 500 & Sectors,% Change
0,S&P 500,1.74
1,Communication Services,0.88
2,Consumer Discretionary,2.47
3,Consumer Staples,1.19
4,Energy,1.45
5,Financials,1.74
6,Health Care,1.89
7,Industrials,2.22
8,Information Technology,1.89
9,Materials,2.62


In [11]:
sp500_sectors_df.to_csv(external+"sp500.csv", index=False) 

In [12]:
sp500_sectors_html = sp500_sectors_df.to_html()
sp500_sectors_table = str(sp500_sectors_html)
sp500_sectors_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>S&amp;P 500 &amp; Sectors</th>\n      <th>% Change</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>S&amp;P 500</td>\n      <td>1.74</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Communication Services</td>\n      <td>0.88</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Consumer Discretionary</td>\n      <td>2.47</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Consumer Staples</td>\n      <td>1.19</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Energy</td>\n      <td>1.45</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Financials</td>\n      <td>1.74</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Health Care</td>\n      <td>1.89</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>Industrials</td>\n      <td>2.22</td>\n    </tr>\n    <tr>\n      <th>8</th>\n      <td>Information Technology</td>\n   

In [13]:
# The index of the links needed
# 0	S&P 500
# 1	Communication Services
# 2	Consumer Discretionary
# 3	Consumer Staples
# 4	Energy
# 5	Financials
# 6	Health Care
# 7	Industrials
# 8	Information Technology
# 9	Materials
# 10	Real Estate
# 11	Utilities
sp500_links = [(driver.find_elements_by_partial_link_text(sector)) for sector in sp500_sectors_df["S&P 500 & Sectors"]]

# variable Links to individual sector pages: 
sp500_link = [links.get_attribute("href") for links in sp500_links[0]]
communication_link = [links.get_attribute("href") for links in sp500_links[1]]
discretionary_link = [links.get_attribute("href") for links in sp500_links[2]]
staples_link = [links.get_attribute("href") for links in sp500_links[3]]
energy_link = [links.get_attribute("href") for links in sp500_links[4]]
financials_link = [links.get_attribute("href") for links in sp500_links[5]]
health_link = [links.get_attribute("href") for links in sp500_links[6]]
industrials_link = [links.get_attribute("href") for links in sp500_links[7]]
information_Technology_link = [links.get_attribute("href") for links in sp500_links[8]]
materials_link = [links.get_attribute("href") for links in sp500_links[9]]
real_estate_link = [links.get_attribute("href") for links in sp500_links[10]]
utilities_link = [links.get_attribute("href") for links in sp500_links[11]]

In [14]:
# list of links for sectors: 
sector_links = [sp500_link, communication_link, discretionary_link, staples_link, energy_link, financials_link, health_link, industrials_link, information_Technology_link, materials_link, real_estate_link, utilities_link]
sector_links

[['https://www.wsj.com/market-data/quotes/index/US/S&P%20US/SPX?mod=md_usstk_overview_quote',
  'https://www.wsj.com/market-data/quotes/futures/ES00?mod=md_usstk_overview_quote',
  'https://www.wsj.com/market-data/quotes/index/US/S&P%20US/SPX?mod=md_usstk_sp_sectors_quote'],
 ['https://www.wsj.com/market-data/quotes/index/XX/S&P%20US/SP500.50?mod=md_usstk_sp_sectors_quote'],
 ['https://www.wsj.com/market-data/quotes/index/XX/S&P%20US/SP500.25?mod=md_usstk_sp_sectors_quote'],
 ['https://www.wsj.com/market-data/quotes/index/XX/S&P%20US/SP500.30?mod=md_usstk_sp_sectors_quote'],
 ['https://www.wsj.com/market-data/quotes/US/BTU?mod=md_usstk_hilo_quote',
  'https://www.wsj.com/market-data/quotes/index/XX/S&P%20US/SP500.10?mod=md_usstk_sp_sectors_quote'],
 ['https://www.wsj.com/market-data/quotes/index/XX/S&P%20US/SP500.40?mod=md_usstk_sp_sectors_quote'],
 ['https://www.wsj.com/market-data/quotes/index/XX/S&P%20US/SP500.35?mod=md_usstk_sp_sectors_quote'],
 ['https://www.wsj.com/market-data/qu

In [15]:
# Get to Historical Data:
driver.get(sp500_link[2])

In [16]:
historical_data = driver.find_elements(By.CSS_SELECTOR, 'a.moreLink')
historical_data

[<selenium.webdriver.remote.webelement.WebElement (session="bfb511137c7e25ea33cd7421c9913302", element="f0550686-ac5f-4779-816a-f75c9d0ae0a5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="bfb511137c7e25ea33cd7421c9913302", element="3bb19f27-0976-489a-a696-71b30e48513b")>]

In [17]:
# Get to historical data download page: 
historical_data[1].click()

In [18]:
# Todays date
currentDate = date.today()
today = currentDate.strftime('%m/%d/%Y')
today

'10/07/2020'

In [19]:
# date 1 week ago from today
five_days = currentDate - timedelta(days=365)
five = five_days.strftime('%m/%d/%Y')
five

'10/08/2019'

In [20]:
# Fill out Date From Form
text_area = driver.find_element(By.CSS_SELECTOR, "#selectDateFrom")
text_area.send_keys(Keys.CONTROL, "a")  # or Keys.COMMAND on Mac
text_area.send_keys(five)

In [21]:
# Fill out Date to
text_area2 = driver.find_element(By.CSS_SELECTOR, "#selectDateTo")
text_area2.send_keys(Keys.CONTROL, "a")  # or Keys.COMMAND on Mac
text_area2.send_keys(today)

In [22]:
# Generate the data
generate_data = driver.find_element(By.ID, "datPickerButton")

In [23]:
generate_data.click()

In [24]:
# Download as csv  
download_sheet = driver.find_element(By.ID, "dl_spreadsheet")

In [25]:
download_sheet.click()

In [ ]:
driver.quit()